In [1]:
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from itertools import combinations
from sklearn.model_selection import KFold
import xgboost as xgb

In [2]:
# https://www.kaggle.com/kyakovlev
# https://www.kaggle.com/competitions/amex-default-prediction/discussion/327534
def amex_metric_mod(y_true, y_pred):

    labels     = np.transpose(np.array([y_true, y_pred]))
    labels     = labels[labels[:, 1].argsort()[::-1]]
    weights    = np.where(labels[:,0]==0, 20, 1)
    cut_vals   = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four   = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])

    gini = [0,0]
    for i in [1,0]:
        labels         = np.transpose(np.array([y_true, y_pred]))
        labels         = labels[labels[:, i].argsort()[::-1]]
        weight         = np.where(labels[:,0]==0, 20, 1)
        weight_random  = np.cumsum(weight / np.sum(weight))
        total_pos      = np.sum(labels[:, 0] *  weight)
        cum_pos_found  = np.cumsum(labels[:, 0] * weight)
        lorentz        = cum_pos_found / total_pos
        gini[i]        = np.sum((lorentz - weight_random) * weight)

    return 0.5 * (gini[1]/gini[0] + top_four)

def xgb_amex_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'amex_metric', amex_metric(y_true, y_pred), True

# **Train XGBoost model**

In [3]:
train = pd.read_parquet('C:\\Users\\16122\\AMEX Kaggle Competition\\train_newnn_fe.parquet')

In [4]:
cols = [col for col in list(train.columns) if col not in ['customer_ID','S_2']]

In [5]:
for col in cols:
    if train[col].dtype == 'int64':
        train[col] = train[col].astype('int8')

In [6]:
cat_features = ["B_30","B_38","D_114","D_116","D_117","D_120","D_126","D_63","D_64","D_66","D_68"]
cat_features = [f"{cf}_last" for cf in cat_features]
for cat_col in cat_features:
    encoder = LabelEncoder()
    train[cat_col] = encoder.fit_transform(train[cat_col])

In [7]:
# Create after-pay features
def after_pay(df):
    for bcol in [f'B_{i}' for i in [1,2,3,4,5,9,11,14,17,24]]+['D_39','D_131']+[f'S_{i}' for i in [16,23]]:
        for pcol in ['P_2','P_3']:
            check = bcol + '_last'
            if check in df.columns:
                df[f'{bcol}-{pcol}_last'] = df[f'{bcol}_last'] - df[f'{pcol}_last']

In [8]:
after_pay(train)

In [9]:
for col in list(train.columns):
    if train[col].dtype == 'float32':
        train[col] = train[col].round(2)

In [10]:
train.head()

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,D_39_last,B_1_mean,B_1_std,B_1_min,B_1_max,B_1_last,B_2_mean,B_2_std,B_2_min,B_2_max,B_2_last,R_1_mean,R_1_std,R_1_min,R_1_max,R_1_last,S_3_mean,S_3_std,S_3_min,S_3_max,S_3_last,D_41_mean,D_41_std,D_41_min,D_41_max,D_41_last,B_3_mean,B_3_std,B_3_min,B_3_max,B_3_last,D_42_mean,D_42_std,D_42_min,D_42_max,D_42_last,D_43_mean,D_43_std,D_43_min,D_43_max,D_43_last,D_44_mean,D_44_std,D_44_min,D_44_max,D_44_last,B_4_mean,B_4_std,B_4_min,B_4_max,B_4_last,D_45_mean,D_45_std,D_45_min,D_45_max,D_45_last,B_5_mean,B_5_std,B_5_min,B_5_max,B_5_last,D_46_mean,D_46_std,D_46_min,D_46_max,D_46_last,D_47_mean,D_47_std,D_47_min,D_47_max,D_47_last,D_48_mean,D_48_std,D_48_min,D_48_max,D_48_last,D_49_mean,D_49_std,D_49_min,D_49_max,D_49_last,B_6_mean,B_6_std,B_6_min,B_6_max,B_6_last,B_7_mean,B_7_std,B_7_min,B_7_max,B_7_last,B_8_mean,B_8_std,B_8_min,B_8_max,B_8_last,D_50_mean,D_50_std,D_50_min,D_50_max,D_50_last,B_9_mean,B_9_std,B_9_min,B_9_max,B_9_last,R_3_mean,R_3_std,R_3_min,R_3_max,R_3_last,D_52_mean,D_52_std,D_52_min,D_52_max,D_52_last,P_3_mean,P_3_std,P_3_min,P_3_max,P_3_last,B_10_mean,B_10_std,B_10_min,B_10_max,B_10_last,D_53_mean,D_53_std,D_53_min,D_53_max,D_53_last,S_5_mean,S_5_std,S_5_min,S_5_max,S_5_last,B_11_mean,B_11_std,B_11_min,B_11_max,B_11_last,D_54_mean,D_54_std,D_54_min,D_54_max,D_54_last,S_7_mean,S_7_std,S_7_min,S_7_max,S_7_last,B_12_mean,B_12_std,B_12_min,B_12_max,B_12_last,S_8_mean,S_8_std,S_8_min,S_8_max,S_8_last,D_55_mean,D_55_std,D_55_min,D_55_max,D_55_last,D_56_mean,D_56_std,D_56_min,D_56_max,D_56_last,B_13_mean,B_13_std,B_13_min,B_13_max,B_13_last,R_5_mean,R_5_std,R_5_min,R_5_max,R_5_last,D_58_mean,D_58_std,D_58_min,D_58_max,D_58_last,S_9_mean,S_9_std,S_9_min,S_9_max,S_9_last,B_14_mean,B_14_std,B_14_min,B_14_max,B_14_last,D_59_mean,D_59_std,D_59_min,D_59_max,D_59_last,D_60_mean,D_60_std,D_60_min,D_60_max,D_60_last,D_61_mean,D_61_std,D_61_min,D_61_max,D_61_last,B_15_mean,B_15_std,B_15_min,B_15_max,B_15_last,S_11_mean,S_11_std,S_11_min,S_11_max,S_11_last,D_62_mean,D_62_std,D_62_min,D_62_max,D_62_last,D_65_mean,D_65_std,D_65_min,D_65_max,D_65_last,B_16_mean,B_16_std,B_16_min,B_16_max,B_16_last,B_17_mean,B_17_std,B_17_min,B_17_max,...,B_27_last_max_ratio,S_17_last_max_ratio,R_12_last_max_ratio,B_28_last_max_ratio,R_13_last_max_ratio,D_83_last_max_ratio,R_14_last_max_ratio,D_84_last_max_ratio,R_16_last_max_ratio,R_17_last_max_ratio,S_19_last_max_ratio,R_20_last_max_ratio,S_22_last_max_ratio,S_23_last_max_ratio,S_24_last_max_ratio,S_25_last_max_ratio,S_26_last_max_ratio,D_102_last_max_ratio,D_104_last_max_ratio,D_105_last_max_ratio,D_106_last_max_ratio,D_107_last_max_ratio,B_36_last_max_ratio,B_37_last_max_ratio,R_26_last_max_ratio,R_27_last_max_ratio,D_112_last_max_ratio,B_40_last_max_ratio,S_27_last_max_ratio,D_113_last_max_ratio,D_115_last_max_ratio,D_118_last_max_ratio,D_119_last_max_ratio,D_121_last_max_ratio,D_122_last_max_ratio,D_124_last_max_ratio,D_128_last_max_ratio,D_130_last_max_ratio,D_131_last_max_ratio,D_133_last_max_ratio,D_141_last_max_ratio,D_142_last_max_ratio,D_144_last_max_ratio,D_145_last_max_ratio,P_2_diff1,D_39_diff1,B_1_diff1,B_2_diff1,R_1_diff1,S_3_diff1,D_41_diff1,B_3_diff1,D_42_diff1,D_43_diff1,D_44_diff1,B_4_diff1,D_45_diff1,B_5_diff1,R_2_diff1,D_46_diff1,D_47_diff1,D_48_diff1,D_49_diff1,B_6_diff1,B_7_diff1,B_8_diff1,D_50_diff1,D_51_diff1,B_9_diff1,R_3_diff1,D_52_diff1,P_3_diff1,B_10_diff1,D_53_diff1,S_5_diff1,B_11_diff1,S_6_diff1,D_54_diff1,R_4_diff1,S_7_diff1,B_12_diff1,S_8_diff1,D_55_diff1,D_56_diff1,B_13_diff1,R_5_diff1,D_58_diff1,S_9_diff1,B_14_diff1,D_59_diff1,D_60_diff1,D_61_diff1,B_15_diff1,S_11_diff1,D_62_diff1,D_65_diff1,B_16_diff1,B_17_diff1,B_18_diff1,B_19_diff1,B_20_diff1,S_12_diff1,R_6_diff1,S_13_diff1,B_21_diff1,D_69_diff1,B_22_diff1,D_70_diff1,D_71_diff1,D_72_diff1,S_15_diff1,B_23_diff1,D_73_diff1,P_4_diff1,D_74_diff1,D_75_diff1,D_76_diff1,B_24_diff1,R_7_diff1,D_77_diff1,B_25_diff1,B_26_diff1,D_78

In [11]:
FEATURES = [col for col in train.columns if col not in ['customer_ID', 'target']]

In [13]:
# XGB MODEL PARAMETERS
xgb_parms = { 
    'max_depth':7, 
    'learning_rate':0.01, 
    'subsample':0.88,
    'eval_metric':'logloss',
    'objective':'binary:logistic',
    'tree_method':'hist',
    'gamma':1.5,
    'min_child_weight':8,
    'lambda':70,
    'eta':0.03,
    'random_state':42}

importances = []
gc.collect()
oof_predictions = np.zeros(len(train))
VER = 0

skf = KFold(n_splits=5, shuffle=True, random_state=42)
for fold,(train_idx, valid_idx) in enumerate(skf.split(train, train.target)):
    print(' ')
    print('-'*50)
    print(f'Training fold {fold} with {len(FEATURES)} features...')
    
    # TRAIN, VALID, TEST FOR FOLD K
    X_train = train.loc[train_idx, FEATURES]
    y_train = train.loc[train_idx, 'target']
    X_valid = train.loc[valid_idx, FEATURES]
    y_valid = train.loc[valid_idx, 'target']
    
    dtrain = xgb.DMatrix(data=X_train, label=y_train)
    dvalid = xgb.DMatrix(data=X_valid, label=y_valid)
    
    # TRAIN MODEL FOLD K
    model = xgb.train(xgb_parms, 
                dtrain=dtrain,
                evals=[(dtrain,'train'),(dvalid,'valid')],
                num_boost_round=9999,
                early_stopping_rounds=500,
                verbose_eval=100) 
    model.save_model(f'xgb_v{VER}_fold{fold}_fea{len(FEATURES)}_round2.xgb')
    
    # GET FEATURE IMPORTANCE FOR FOLD K
    dd = model.get_score(importance_type='weight')
    df = pd.DataFrame({'feature':dd.keys(),f'importance_{fold}':dd.values()})
    importances.append(df)
            
    # Stroe prediction of validation data
    oof_preds = model.predict(dvalid, iteration_range=(0, model.best_ntree_limit)) 
    oof_predictions[valid_idx] = np.array(oof_preds)
    score = amex_metric_mod(y_valid.values, oof_preds)
    print(f'Our fold {fold} CV score is {score}')
  
    del dtrain, dvalid, dd, df
    del X_valid, y_valid, model
    _ = gc.collect()
    
print('_'*50)
# Create a dataframe to save out of folds predictions
overall_score = amex_metric_mod(train['target'], oof_predictions)
oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train['target'], 'prediction': oof_predictions})
oof_df.to_csv(f'oof_xgb_v{VER}_{5}fold_seed{42}_fea{len(FEATURES)}_round2.csv', index = False)
print(f'Our out of folds CV score is {overall_score}')

 
--------------------------------------------------
Training fold 0 with 1285 features...
[0]	train-logloss:0.68665	valid-logloss:0.68664
[100]	train-logloss:0.36251	valid-logloss:0.36503
[200]	train-logloss:0.27240	valid-logloss:0.27729
[300]	train-logloss:0.24059	valid-logloss:0.24741
[400]	train-logloss:0.22718	valid-logloss:0.23562
[500]	train-logloss:0.22029	valid-logloss:0.23024
[600]	train-logloss:0.21566	valid-logloss:0.22722
[700]	train-logloss:0.21202	valid-logloss:0.22524
[800]	train-logloss:0.20909	valid-logloss:0.22380
[900]	train-logloss:0.20664	valid-logloss:0.22276
[1000]	train-logloss:0.20446	valid-logloss:0.22192
[1100]	train-logloss:0.20239	valid-logloss:0.22120
[1200]	train-logloss:0.20051	valid-logloss:0.22057
[1300]	train-logloss:0.19874	valid-logloss:0.22007
[1400]	train-logloss:0.19707	valid-logloss:0.21962
[1500]	train-logloss:0.19551	valid-logloss:0.21926
[1600]	train-logloss:0.19403	valid-logloss:0.21893
[1700]	train-logloss:0.19266	valid-logloss:0.21868
[18

[1500]	train-logloss:0.19649	valid-logloss:0.21589
[1600]	train-logloss:0.19501	valid-logloss:0.21557
[1700]	train-logloss:0.19355	valid-logloss:0.21525
[1800]	train-logloss:0.19215	valid-logloss:0.21500
[1900]	train-logloss:0.19082	valid-logloss:0.21479
[2000]	train-logloss:0.18950	valid-logloss:0.21458
[2100]	train-logloss:0.18824	valid-logloss:0.21441
[2200]	train-logloss:0.18701	valid-logloss:0.21423
[2300]	train-logloss:0.18582	valid-logloss:0.21408
[2400]	train-logloss:0.18461	valid-logloss:0.21392
[2500]	train-logloss:0.18355	valid-logloss:0.21380
[2600]	train-logloss:0.18241	valid-logloss:0.21367
[2700]	train-logloss:0.18131	valid-logloss:0.21355
[2800]	train-logloss:0.18020	valid-logloss:0.21345
[2900]	train-logloss:0.17912	valid-logloss:0.21336
[3000]	train-logloss:0.17811	valid-logloss:0.21328
[3100]	train-logloss:0.17702	valid-logloss:0.21320
[3200]	train-logloss:0.17602	valid-logloss:0.21313
[3300]	train-logloss:0.17507	valid-logloss:0.21304
[3400]	train-logloss:0.17405	va

[2600]	train-logloss:0.18184	valid-logloss:0.21822
[2700]	train-logloss:0.18078	valid-logloss:0.21811
[2800]	train-logloss:0.17969	valid-logloss:0.21800
[2900]	train-logloss:0.17868	valid-logloss:0.21790
[3000]	train-logloss:0.17762	valid-logloss:0.21780
[3100]	train-logloss:0.17658	valid-logloss:0.21771
[3200]	train-logloss:0.17558	valid-logloss:0.21763
[3300]	train-logloss:0.17449	valid-logloss:0.21754
[3400]	train-logloss:0.17350	valid-logloss:0.21747
[3500]	train-logloss:0.17247	valid-logloss:0.21741
[3600]	train-logloss:0.17148	valid-logloss:0.21736
[3700]	train-logloss:0.17053	valid-logloss:0.21730
[3800]	train-logloss:0.16952	valid-logloss:0.21725
[3900]	train-logloss:0.16858	valid-logloss:0.21721
[4000]	train-logloss:0.16755	valid-logloss:0.21717
[4100]	train-logloss:0.16662	valid-logloss:0.21712
[4200]	train-logloss:0.16570	valid-logloss:0.21709
[4300]	train-logloss:0.16481	valid-logloss:0.21705
[4400]	train-logloss:0.16387	valid-logloss:0.21703
[4500]	train-logloss:0.16296	va

In [ ]:
oof_df.head()

In [ ]:
# CLEAN RAM
del train
_ = gc.collect()

# **Prediction and submission**

In [ ]:
test = pd.read_parquet('C:\\Users\\16122\\AMEX Kaggle Competition\\test_newnn_fe.parquet')

In [ ]:
cols = [col for col in list(test.columns) if col not in ['customer_ID','S_2']]

In [ ]:
for col in cols:
    if test[col].dtype == 'int64':
        test[col] = test[col].astype('int8')

In [ ]:
cat_features = ["B_30","B_38","D_114","D_116","D_117","D_120","D_126","D_63","D_64","D_66","D_68"]
cat_features = [f"{cf}_last" for cf in cat_features]
for cat_col in cat_features:
    encoder = LabelEncoder()
    test[cat_col] = encoder.fit_transform(test[cat_col])

In [ ]:
after_pay(test)

In [ ]:
for col in list(test.columns):
    if test[col].dtype == 'float32':
        test[col] = test[col].round(2)

In [ ]:
FEATURES = [col for col in test.columns if col not in ['customer_ID', 'target']]

In [ ]:
def predict_test(test, features):
    # Create a numpy array to store test predictions
    test_predictions = np.zeros(len(test))
    X_test = test[features]
    dtest = xgb.DMatrix(X_test)
    for fold in range(5):
        # Predict the test set
        print(f'=========== Fold {fold} is predicting ===========')
        model = xgb.Booster()
        model.load_model(f'xgb_v{VER}_fold{fold}_fea{len(features)}_round2.xgb')
        test_pred = model.predict(dtest)
        test_predictions += test_pred / 5
    # Create a dataframe to save test prediction
    test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
    test_df.to_csv(f'XGB_{5}folds_seed{42}_fe{len(features)}_round2.csv', index = False)

In [ ]:
predict_test(test,FEATURES)

In [ ]:
# double check the final result
pred_df = pd.read_csv(f'XGB_{5}folds_seed{42}_fe{len(FEATURES)}_round2.csv')

In [ ]:
pred_df.head()